In [1]:
import warnings
from io import BytesIO
from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile

import pandas as pd
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from sklearn.metrics import brier_score_loss, roc_auc_score
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [2]:
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)


In [3]:
data_files = {
    'events': 'https://ndownloader.figshare.com/files/14464685',  # ZIP file containing one JSON file for each competition
    'matches': 'https://ndownloader.figshare.com/files/14464622',  # ZIP file containing one JSON file for each competition
    'players': 'https://ndownloader.figshare.com/files/15073721',  # JSON file
    'teams': 'https://ndownloader.figshare.com/files/15073697'  # JSON file
}

In [4]:
for url in tqdm(data_files.values()):
    url_s3 = urlopen(url).geturl()
    path = Path(urlparse(url_s3).path)
    file_name = path.name
    file_local, _ = urlretrieve(url_s3, file_name)
    if is_zipfile(file_local):
        with ZipFile(file_local) as zip_file:
            zip_file.extractall()

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

In [6]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [26]:
df_teams.to_hdf('wyscout.h5', key='teams', mode='w')


In [12]:
#df_temp = pd.read_json('./../../data/wyscout/Teams/teams.json')

In [31]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [32]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')
#que hay que descargar otra cosa

In [35]:
competitions = [
     'England',
     'France',
     'Germany',
     'Italy',
    'Spain',
    'European Championship',
    'World Cup'
]

In [36]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [37]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

In [38]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

# Change features

In [39]:
convert_to_spadl('wyscout.h5', 'spadl.h5')


...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players
...Converting teams
...Generating player_games


100%|████████████████████████████████████████████████████████████████████████████| 1941/1941 [02:59<00:00, 10.81game/s]


...Converting events to actions


100%|████████████████████████████████████████████████████████████████████████████| 1941/1941 [18:13<00:00,  1.77game/s]


In [40]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [41]:
nb_prev_actions = 3


In [42]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

In [43]:

for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

  0%|          | 0/1941 [00:00<?, ?it/s]

In [44]:
functions_labels = [
    labels.scores,
    labels.concedes
]

In [45]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

  0%|          | 0/1941 [00:00<?, ?it/s]

## Generate dataset

In [46]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)


In [47]:

dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

  0%|          | 0/1941 [00:00<?, ?it/s]

In [49]:
columns_labels = [
    'scores',
    'concedes'
]

In [50]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

  0%|          | 0/1941 [00:00<?, ?it/s]

# Train data

In [51]:

%%time
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(
        eval_metric='logloss',
        use_label_encoder=False
    )
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

C:\Users\Ernie\anaconda3\envs\sport_analysis\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


CPU times: total: 3h 31min 37s
Wall time: 15min 47s


In [68]:
#models['scores'].save_model("model_scores.json")
#models['concedes'].save_model("model_concedes.json")

In [52]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [53]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

  0%|          | 0/1941 [00:00<?, ?it/s]

In [54]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)


In [55]:
df_predictions_per_game = df_predictions.groupby('game_id')


In [56]:

for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

  0%|          | 0/1941 [00:00<?, ?it/s]

# Value on-the-ball action

In [57]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [58]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

  0%|          | 0/1941 [00:00<?, ?it/s]

In [74]:
#df_values.to_csv('xgboost_vaep.csv')

In [59]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [61]:
df_values[
    ['short_name', 'scores', 'concedes', 'offensive_value', 'defensive_value', 'vaep_value']
]

,short_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,O. Giroud,0.003433,0.000746,0.000000,-0.000000,0.000000
1,A. Griezmann,0.004674,0.003075,0.001241,-0.002329,-0.001088
2,N. Kanté,0.004724,0.002211,0.000050,0.000864,0.000914
3,L. Koscielny,0.003231,0.002185,-0.001493,0.000026,-0.001468
4,P. Evra,0.006811,0.001348,0.006811,-0.001348,0.005463
...,...,...,...,...,...,...
2465151,Pedro Pereira,0.006680,0.003939,-0.014889,-0.000505,-0.015394
2465152,D. Laxalt,0.015082,0.002576,0.015082,-0.002576,0.012506
2465153,A. Belotti,0.003039,0.053981,0.000463,-0.038899,-0.038436
2465154,A. Belotti,0.003260,0.034843,0.000221,0.019138,0.019359


In [69]:
df_values.columns #sort_values(by='vaep_value')

Index(['game_id', 'period_id', 'time_seconds', 'team_id', 'player_id',
       'start_x', 'start_y', 'end_x', 'end_y', 'bodypart_id', 'type_id',
       'result_id', 'type_name', 'result_name', 'bodypart_name', 'short_name',
       'first_name', 'last_name', 'birth_date', 'short_team_name', 'team_name',
       'scores', 'concedes', 'offensive_value', 'defensive_value',
       'vaep_value'],
      dtype='object')

# Ratings

In [75]:
df_ranking = (df_values[['player_id', 'team_name', 'short_name', 'vaep_value']]
    .groupby(['player_id', 'team_name', 'short_name'])
    .agg(vaep_count=('vaep_value', 'count'), vaep_sum=('vaep_value', 'sum'))
    .sort_values('vaep_sum', ascending=False)
    .reset_index()
)

In [77]:
df_ranking.head(20)

,player_id,team_name,short_name,vaep_count,vaep_sum
0,3359.0,FC Barcelona,L. Messi,2753,36.898487
1,120353.0,Liverpool FC,Mohamed Salah,1568,25.638689
2,40810.0,Paris Saint-Germain FC,Neymar,1981,20.012957
3,3682.0,Club Atlético de Madrid,A. Griezmann,1394,18.764013
4,38021.0,Manchester City FC,K. De Bruyne,3528,18.684515
5,3840.0,Real Club Celta de Vigo,Iago Aspas,1771,18.659481
6,26150.0,Leicester City FC,R. Mahrez,2022,18.633902
7,8717.0,Tottenham Hotspur FC,H. Kane,1153,18.459000
8,25770.0,Olympique de Marseille,F. Thauvin,2511,18.438290
9,89186.0,Juventus FC,P. Dybala,1782,17.917562


In [78]:
df_player_games = pd.read_hdf('spadl.h5', 'player_games')
df_player_games = df_player_games[df_player_games['game_id'].isin(df_games['game_id'])]

In [79]:
df_minutes_played = (df_player_games[['player_id', 'minutes_played']]
    .groupby('player_id')
    .sum()
    .reset_index()
)

In [80]:
df_ranking_p90 = df_ranking.merge(df_minutes_played)
df_ranking_p90 = df_ranking_p90[df_ranking_p90['minutes_played'] > 360]
df_ranking_p90['vaep_rating'] = df_ranking_p90['vaep_sum'] * 90 / df_ranking_p90['minutes_played']
df_ranking_p90 = df_ranking_p90.sort_values('vaep_rating', ascending=False)

In [82]:
df_ranking_p90.head(30)


,player_id,team_name,short_name,vaep_count,vaep_sum,minutes_played,vaep_rating
0,3359.0,FC Barcelona,L. Messi,2753,36.898487,3486.545778,0.952480
4,40810.0,Paris Saint-Germain FC,Neymar,1981,20.012957,2334.354641,0.771591
2,120353.0,Liverpool FC,Mohamed Salah,1568,25.638689,3186.109293,0.724232
17,89186.0,Juventus FC,P. Dybala,1782,17.917562,2477.563907,0.650873
606,406682.0,Rasen Ballsport Leipzig,A. Lookman,345,5.374696,771.082910,0.627329
123,134397.0,TSG 1899 Hoffenheim,S. Gnabry,836,10.304981,1534.906306,0.604238
1352,326523.0,Real Madrid Club de Fútbol,Dani Ceballos,446,2.823848,420.702252,0.604100
22,135913.0,AS Monaco FC,Rony Lopes,1304,17.417822,2596.649231,0.603703
19,28115.0,Olympique Lyonnais,N. Fekir,1754,17.818094,2671.557291,0.600260
21,118.0,Olympique Lyonnais,M. Depay,1837,17.532991,2634.701474,0.598918
